Analisando a página da Drogaria Minas Mais, percebo que ela é uma página dinâmica (tem o botão de "Carregar mais produtos")
Outro fator que também chamou atenção: ao desligar o Javascript da página, ela não exibiu nenhum produto (sequer conseguiu carregar), exibindo uma dependenciaforte do JS
Assim, vou tentar uma abordagem para um HTML dinâmico 



No site há 6118 produtos, sendo que na primeira página há 15 produtos, na segunda há outros 15 produtos e assim por diante 
Temos um total de 408 páginas
Entretanto na página 51 já não encontramos mais produtos (Acredito que o acesso esteja limitado pelo parâmetro Page, como previa o arquivo de treinamento interno do Notion) 


Vou tentar fazer o processo de filtrar por marcas
A marca com mais produtos é a HYPERA com 371 produtos
O layout do site apresenta 15 produtos por página. Como podemos acessar 50 páginas, temos acesso a 750 (50x15) produtos no máximo
Assim filtrar por marca realmente será uma estratégia válida

<label class="vtex-checkbox__label w-100 c-on-base pointer" for="brand-hypera">HYPERA</label>
<label class="vtex-checkbox__label w-100 c-on-base pointer" for="brand-ache-rx">ACHE RX</label>

Cliquei em inspect em cima das marcas e achei isso no código fonte.
Dar um jeito de pegar todas as marcas com essa estrutura

In [22]:
from bs4 import BeautifulSoup
import requests

In [23]:
def pegar_html(link):  
    req=requests.get(link)
    html=BeautifulSoup(req.text,"html.parser")
    return html
html=pegar_html(link="https://www.drogariasminasmais.com.br/medicamentos?initialMap=c&initialQuery=medicamentos&map=category-1")

In [24]:
def achar_marcas(html):
    
    marcas=html.find_all("label", attrs={"class":"vtex-checkbox__label w-100 c-on-base pointer"})

    strmarcas=[]
    POP=[]
    REMOVER=[]

    #transformar ps códigos <label ...  em strings
    for elem in marcas:
        strmarcas.append(str(elem))

    #achar os índices das labels(em string) que possuem for="category-2...", que não representam marcas
    for i in range(len(strmarcas)):
        if "category-2" in strmarcas[i]:
            POP.append(i)

    #Achar a correlação entre as labels(em string) e as labels normais. Saber qual label normal deve ser removida
    for indice in POP:
        REMOVER.append(marcas[indice])

    for indesejado in REMOVER:
        marcas.remove(indesejado)

    #pegando somente o nome das marcas (retirando toda a estrutura do label)
    TEXTO=[]
    for label in marcas:
        TEXTO.append(label.string)

    #analisando o sorted(TEXTO), percebi que diversos elementos da lista eram duplicados

    #Tirando os nomes duplicados
    TEXTO_S_DUPL=[]
    for elem in TEXTO:
        if elem not in TEXTO_S_DUPL:
            TEXTO_S_DUPL.append(elem) 
    return TEXTO_S_DUPL



marcas=achar_marcas(html)
marcas


['HYPERA',
 'ACHE RX',
 'EUROFARMA RX',
 'EMS GENERICO',
 'CIMED',
 'SIGMA PHARMA RX',
 'GERMED GENERICO',
 'BIOLAB RX',
 'LIBBS',
 'APSEN',
 'MEDLEY GENERICOS',
 'SUPERA',
 'FARMOQUIMICA',
 'EUROFARMA GENERICO',
 'MOMENTA',
 'MYRALIS',
 'ABBOTT RX',
 'TEUTO GENERICO',
 'ACHE OTC',
 'SANOFI RX',
 'CRISTALIA/LATINOFARMA',
 'TORRENT',
 'PRATI DONADUZZI GENERICO',
 'SANOFI OTC',
 'UNIAO QUIMICA GENOM',
 'NEO QUIMICA GENERICO',
 'UNIAO QUIMICA SIMILAR',
 'PHARLAB',
 'BIOLAB GENERICO',
 'NEO QUIMICA SIMILAR',
 'LEGRAND GENERICO',
 'GSK OTC',
 'MARJAN',
 'NOVARTIS RX',
 'NOVA QUIMICA GENERICO',
 'ZANPHY',
 'GEOLAB GENERICO',
 'GSK RX',
 'PROCTER  GAMBL',
 'UNIAO QUIMICA GENERICO',
 'MERCK RX',
 'BIOSINTETICA GENERICO',
 'ALLERGAN',
 'ALCON',
 'BAYER RX',
 'TEUTO SIMILAR',
 'SERVIER',
 'HERBARIUM',
 'GLOBO GENERICO',
 'CELLERA',
 'BIOLAB AVERT OTC',
 'ASTRAZENECA',
 'SANDOZ GENERICO',
 'EMS SIMILAR',
 'DIVCOM',
 'SANKYO PHARMA',
 'RANBAXY',
 'BAYER OTC',
 'ZODIAC ADIUM',
 'BOEHRINGER',
 'AIRE

In [25]:
def marca_link(M):
    MARCA_PARA_LINK=[]
    for elem in M:
        elem=elem.replace(" ","-")
        elem=elem.replace("/","-")
        elem=elem.lower()
        MARCA_PARA_LINK.append(elem)
    return MARCA_PARA_LINK


marcas_formatadas=marca_link(marcas)

In [26]:
def link (MARCA_LINK):
    LINK=[]
    for i in MARCA_LINK:
        link=f"https://www.drogariasminasmais.com.br/medicamentos/{i}?initialMap=c&initialQuery=medicamentos&map=category-1,brand"
        link+="&page=1"
        LINK.append(link)
    return(LINK)

LINKS=link(marcas_formatadas)
LINKS

['https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1',
 'https://www.drogariasminasmais.com.br/medicamentos/ache-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1',
 'https://www.drogariasminasmais.com.br/medicamentos/eurofarma-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1',
 'https://www.drogariasminasmais.com.br/medicamentos/ems-generico?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1',
 'https://www.drogariasminasmais.com.br/medicamentos/cimed?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1',
 'https://www.drogariasminasmais.com.br/medicamentos/sigma-pharma-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1',
 'https://www.drogariasminasmais.com.br/medicamentos/germed-generico?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1',
 'https://www.drogariasminasmais.com.br/medicamentos/biolab-rx?init

In [27]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

service=Service(ChromeDriverManager().install())
navegador=webdriver.Chrome(service=service)


In [28]:
navegador.get('https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1')